## Running Vitis HLS from Jupyter

This cell executes a Vitis HLS synthesis flow directly from a Jupyter Notebook, allowing the automatic generation of an HLS project and its corresponding IP core using a TCL script.

The approach enables seamless integration of HLS-based hardware generation within a Python-based workflow, which is especially useful when working with tools such as hls4ml, automated design pipelines, or reproducible ML-to-FPGA flows.

In [11]:
# Import required modules
from pathlib import Path
import subprocess
import os
import shutil
import zipfile



In [12]:
# Configure the Vitis HLS environment
# This line prepends the Vitis HLS binary directory to the system PATH, ensuring that the vitis_hls executable can be called from within the notebook.

# THIS IS YOUR INSTALLATION PATH -> SHOULD BE MODIFIED ACCORDINGLY
os.environ['PATH'] = '/tools/Xilinx/XilinxUnified_2022/Vitis_HLS/2022.2/bin:' + os.environ['PATH']

os.environ['PATH']



'/tools/Xilinx/XilinxUnified_2022/Vitis_HLS/2022.2/bin:/tools/Xilinx/XilinxUnified_2022/Vitis_HLS/2022.2/bin:/tools/anaconda3/envs/neuralEnv/bin:/tools/anaconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin'

In [13]:
# ------------------------------------------------------------------
# Define the HLS project directory
# ------------------------------------------------------------------
# PATH_HLS_PROJECT must point to the root directory of the HLS project
# generated by KalEdge-Lite.
#
# This directory is created after extracting the HLS project archive
# downloaded from the KalEdge-Lite app.
# ------------------------------------------------------------------

PATH_HLS_PROJECT = '../../tmp_files/hls4ml_best_model_comblock/hls4ml_best_model'

PROJ_DIR = Path(PATH_HLS_PROJECT)



In [14]:
# Execute the HLS build script
res = subprocess.run(
    ["vitis_hls", "-f", "build_myproject_bincls_axilite.tcl"],
    cwd=PROJ_DIR,                      
    stdout=subprocess.PIPE, 
    stderr=subprocess.STDOUT,
    text=True,
)

# Display execution results
print("Return code:", res.returncode)
print(res.stdout)

Return code: 0
/bin/bash: /tools/anaconda3/envs/neuralEnv/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /tools/anaconda3/envs/neuralEnv/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /tools/anaconda3/envs/neuralEnv/lib/libtinfo.so.6: no version information available (required by /bin/bash)

****** Vitis HLS - High-Level Synthesis from C, C++ and OpenCL v2022.2 (64-bit)
  **** SW Build 3670227 on Oct 13 2022
  **** IP Build 3669848 on Fri Oct 14 08:30:02 MDT 2022
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.

source /tools/Xilinx/XilinxUnified_2022/Vitis_HLS/2022.2/scripts/vitis_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/tools/Xilinx/XilinxUnified_2022/Vitis_HLS/2022.2/bin/unwrapped/lnx64.o/vitis_hls'
INFO: [HLS 200-10] For user 'ro' on host 'mareKaleido' (Linux_x86_64 version 5.15.0-139-generic) on Fri Jan 23 01:39:57 CET 2026
INFO: [HLS 200-10] On os Ubuntu 20.04.6 LTS
INFO: [HLS 200

In [17]:
from helpers import parse_csynth_xml, hls_report_to_dataframe,csynth_xml_path, list_impl_ip, csynth_xml_from_project_root, export_hls_ip

report = parse_csynth_xml(csynth_xml_path(PATH_HLS_PROJECT))
# print(report)
df = hls_report_to_dataframe(report)
print("HLS report:")
display(df)


HLS report:


,Used,Available,Util (%)
LUT,43203,70560,61.23
FF,14379,141120,10.19
DSP,86,360,23.89
BRAM_18K,2,432,0.46
URAM,0,0,NaN


In [18]:
path2  = Path(PATH_HLS_PROJECT) / 'hls4ml_best_model_accel_prj'
ip_dir, zips, dirs = list_impl_ip(path2)
print("impl/ip:", ip_dir)
print("zips:", [z.name for z in zips])
print("dirs:", [d.name for d in dirs])

print("csynth:", csynth_xml_from_project_root(path2))


impl/ip: ../../tmp_files/hls4ml_best_model_comblock/hls4ml_best_model/hls4ml_best_model_accel_prj/solution1/impl/ip
zips: ['xilinx_com_hls_myproject_bincls_axilite_1_0.zip']
dirs: ['.Xil', 'constraints', 'doc', 'example', 'hdl', 'misc', 'subcore', 'xgui']
csynth: ../../tmp_files/hls4ml_best_model_comblock/hls4ml_best_model/hls4ml_best_model_accel_prj/solution1/syn/report/csynth.xml


In [19]:

info = export_hls_ip(path2)

print("ZIP copied to:", info["zip"])
print("IP extracted at:", info["ip_dir"])

ZIP copied to: ip/hls4ml_best_model_accel_prj/xilinx_com_hls_myproject_bincls_axilite_1_0.zip
IP extracted at: ip/hls4ml_best_model_accel_prj


## Automated Vivado Execution via TCL Scripts

This notebook provides a utility function to execute Vivado TCL scripts in batch mode directly from Python.
It enables automation of synthesis and implementation flows by invoking Vivado programmatically, capturing console output, and handling execution errors.
This approach is particularly useful for reproducible FPGA workflows and integration within larger ML-to-hardware pipelines.

In [20]:
def run_vivado_tcl(tcl_file_path, vivado_path="vivado"):

    """
    Executes a Vivado .tcl file for synthesis.

    tcl_file_path: Path to the .tcl file to be executed.
    vivado_path: Path to the Vivado executable (default assumes it's in PATH).
    """

    # ---- User should specify Vivado installation path
    # THIS IS YOUR INSTALLATION PATH -> SHOULD BE MODIFIED ACCORDINGLY
    vivado_path = '/tools/Xilinx/XilinxUnified_2022/Vivado/2022.2/bin/vivado'
    # ----
    
    try:
        # Command to execute Vivado in batch mode
        command = [vivado_path, "-mode", "batch", "-source", tcl_file_path]

        # Run the command and capture output
        result = subprocess.run(
            command, 
            stdout=subprocess.PIPE, 
            stderr=subprocess.PIPE,     
            text=True
        )

        # Check if the command was successful
        if result.returncode == 0:
            print("Vivado .tcl script executed successfully!")
            print("Output:\n", result.stdout)
        else:
            print("Error during Vivado execution.")
            print("Error message:\n", result.stderr)

    except FileNotFoundError:
        print(f"Error: Vivado executable not found at '{vivado_path}'.")
    except Exception as e:
        print(f"Unexpected error: {e}")

In [21]:
# Commnet out the desired tcl file

# 4eg-21
# tcl_file_path = 'tcl/ML_4eg_21_kaledgeLite.tcl' 

# 3be-11
tcl_file_path = 'tcl/ML_3be_11_kaledgeLite.tcl' 

run_vivado_tcl(tcl_file_path)

Vivado .tcl script executed successfully!
Output:
 
****** Vivado v2022.2 (64-bit)
  **** SW Build 3671981 on Fri Oct 14 04:59:54 MDT 2022
  **** IP Build 3669848 on Fri Oct 14 08:30:02 MDT 2022
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.

source tcl/ML_3be_11_kaledgeLite.tcl
# namespace eval _tcl {
# proc get_script_folder {} {
#    set script_path [file normalize [info script]]
#    set script_folder [file dirname $script_path]
#    return $script_folder
# }
# }
# variable script_folder
# set script_folder [_tcl::get_script_folder]
# set scripts_vivado_version 2022.2
# set current_vivado_version [version -short]
# if { [string first $scripts_vivado_version $current_vivado_version] == -1 } {
#    puts ""
#    catch {common::send_gid_msg -ssname BD::TCL -id 2041 -severity "ERROR" "This script was generated using Vivado <$scripts_vivado_version> and is being run in <$current_vivado_version> of Vivado. Please run the script in Vivado <$scripts_vivado_version> then open t